In [ ]:
load_trained = True # Carrega do drive os dados do modelo salvo no último treinamento
save_after_each_epoch = True # Salva o modelo a cada epoch concluída
epochs = 1 # Quantidade de epochs do treinamento
colab_enviroment = True # Se estiver rodando no colab, setar como True

In [ ]:
if colab_enviroment:

    # mount google drive
    from google.colab import drive
    drive.mount('/content/drive')

    # unzip dataset
    !unzip -q "/content/drive/MyDrive/FSI/dataset/dataset.zip" -d "/dataset"

    if load_trained:
        !cp -r /content/drive/MyDrive/FSI/saved_models /saved_models

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import tensorflow as tf
import itertools
import sklearn.metrics as metrics
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
#Need to edit below import if using base model other than DenseNet
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.optimizers import Adam, SGD
#Need to edit below import if using base model other than DenseNet
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

#Setting seeds so that randomization is kept consistent.
np.random.seed(70) 
random.seed(70) 
tf.random.set_seed(70)

#CSV logger automatically keeps track of accuracy and loss for both training and validation during each epoch.
#This is very convenient for graphing model performance through time later in R
#Outputs a .csv file
log_csv = CSVLogger('my_logs_robocrop.csv', separator=',', append = False)
callbacks_list = [log_csv]

#This is the dimensions that each image will be shaped to
#DenseNet201 requires 224 x 224.
img_height, img_width = (224,224)
#Batch size is the number of training examples utilized in one iteration. Could use 16, may increase compute time.
batch_size = 32

#These are the directories/folders where your images are stored for training, validation, and test datasets.
train_data_dir = r"dataset/train"
valid_data_dir = r"dataset/test"
test_data_dir = r"dataset/val"

#ImageDataGenerator performs image augmentation for each image on the fly. Rotating, flipping, brightness, etc.
train_datagen = ImageDataGenerator(
    width_shift_range= 0.2, # 0.2 fraction of total width/height
    height_shift_range=0.2,
    fill_mode="nearest",
    brightness_range=[0.9,1.1], #range for picking a shift value from
    rotation_range =30, #degree range for random rotations
    vertical_flip = True,
    horizontal_flip = True,
    validation_split = 0.05,
    rescale=1./255) #rescaling image pixel values by the number of channels, 1/255

#Pulls your training dataset images
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical' #more than 2 classes --> binary
    ) #set as training data

#Pulls your validation dataset images
valid_generator = train_datagen.flow_from_directory(
    valid_data_dir, #same directory as training data
    target_size = (img_height, img_width),
    batch_size=batch_size,
    class_mode= 'categorical') #set as validation data

#Pulls your test dataset images. Note that you only want to use 1 image at a time for test.
test_generator = train_datagen.flow_from_directory(
    test_data_dir, #same directory as training data
    target_size = (img_height, img_width),
    batch_size=1,
    class_mode= 'categorical') #set as validation data



Found 230 images belonging to 3 classes.
Found 73 images belonging to 3 classes.
Found 84 images belonging to 3 classes.


In [ ]:
if load_trained:
  # load previous trained model
  model = keras.models.load_model('saved_models/dense_1118.h5')
else:
  #Loads array and classes of items in test folder for each batch
  # x,y = test_generator.next()
  # x.shape
  # print("Xshape:", x.shape)
  # print("n_classes: ", valid_generator.num_classes)
  # print("n_classes: ", test_generator.num_classes)
  # print("y: ", y)

  #Sets base model architecture, could swap out DenseNet201 for ResNet50, VGG16, etc
  # Whether to include the fully-connected layer at top of network
  # 'imagenet' (pre-training on ImageNet), or the path to the weights file to be loaded.
  base_model = DenseNet201(include_top=False, weights='imagenet')
  #Below x lines are the additional architecture attached beyond the base model
  x = base_model.output
  #Globalaveragepooling performs the 'flatten' purposes
  # x = Flatten()(x)
  x = GlobalAveragePooling2D()(x)
  #128, 64 neuron fully-connected layers with relu activation
  #Dropout for regularization
  x = Dense(128, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(64, activation='relu')(x)
  x = Dropout(0.3)(x)
  #Predicting a label for each image based on softmax activation, for 8 classes
  predictions = Dense(train_generator.num_classes, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)

  #Deciding whether to freeze the base model weights (imagenet) or allow them to update
  #Setting this to 'True' implies we are training the entire model
  for layer in base_model.layers:
      layer.trainable = True

  #Compile the model using stochastic gradient descent w/ learning rate and momentum values
  #Use categorical crossentropy as the loss function
  model.compile(optimizer=SGD(learning_rate=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

if save_after_each_epoch:
  model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
      filepath='saved_models/dense_1118.h5',
      save_weights_only=False,
      verbose=1)

  model_checkpoint_callback_best = tf.keras.callbacks.ModelCheckpoint(
      filepath='saved_models/best_val_acc.h5',
      save_weights_only=False,
      monitor='val_accuracy',
      mode='max',
      save_best_only=True,
      initial_value_threshold=0.9,
      verbose=1)
  
  callbacks_list.append(model_checkpoint_callback)
  callbacks_list.append(model_checkpoint_callback_best)

#Train the model for 100 epochs, verbose just tells you much info to display during training
model.fit(train_generator, epochs = epochs, verbose=1, validation_data = valid_generator, callbacks=callbacks_list)

#Add the below block if you want to experiment with freezing the base weights for the above epochs.
# Note, would need to change above to layer.trainable=False to freeze the above epochs.
# Would likely change the learning rate above too.
# And then unfreeze them for additional epochs below. (layer.trainable=True) 
"""
for layer in base_model.layers:
    layer.trainable = True

model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(train_generator, epochs = 10, verbose=1, validation_data = valid_generator, callbacks=callbacks_list)"""

#The big block below lays out the steps for calculating and plotting
# a classification report (precision, recall, etc.) and confusion matrix 
# I prefer to plot manually using wcipriano's pretty print confusion matrix

test_steps_per_epoch = np.math.ceil(valid_generator.samples / valid_generator.batch_size)
Y_pred = model.predict(valid_generator, steps= test_steps_per_epoch)
true_classes = valid_generator.classes
predicted_classes = np.argmax(Y_pred, axis=1)
class_labels = list(valid_generator.class_indices.keys())
cm1 = confusion_matrix(true_classes, predicted_classes)
print(cm1)

print('Classification Report')
report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
print(report)

def plot_confusion_matrix(cm, classes,
                        normalize=False,
                        title='Confusion matrix',
                        cmap=plt.cm.Blues):
   
    #This function prints and plots the confusion matrix.
    #Normalization can be applied by setting `normalize=True`
    
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
            horizontalalignment="center",
            color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

#Compute Confusion Matrix
cnf_matrix = confusion_matrix(true_classes, predicted_classes)
cm_plot_labels = ['bacterial_blight', 'cercospora_leaf_blight','downey_mildew', 'frogeye', 'non_disease', 'potassium_deficiency', 'soybean_rust', 'target_spot']
np.set_printoptions(precision=2)
#Plot non-normalized cm
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cm_plot_labels, title='Confusion Matrix, Without Normalization')
plt.savefig("without_normalized.png")

#Plot normalized cm
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=cm_plot_labels, normalize=True, title='Normalized Confusion Matrix')
plt.savefig("normalized.png")


#Evaluate the performance of the model using test_generator
test_loss, test_acc = model.evaluate(test_generator, verbose=2)
print('\nTest Accuracy:', test_acc)

#Save the model and weights
#Mainly need the .h5 file to best hosted on the webapp
model.save_weights('saved_models/dense_weights_1118')
model.save('saved_models/dense_1118.hdf5')
if not save_after_each_epoch:
  model.save('saved_models/dense_1118.h5')

In [ ]:
 model = keras.models.load_model('/content/drive/MyDrive/saved_model/best_val_acc.h5')



 test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
Y_pred = model.predict(test_generator, steps= test_steps_per_epoch)
true_classes = test_generator.classes
predicted_classes = np.argmax(Y_pred, axis=1)
class_labels = list(test_generator.class_indices.keys())
cm1 = confusion_matrix(true_classes, predicted_classes)
print(cm1)

# print('Classification Report')
# report = metrics.classification_report(true_classes, predicted_classes, target_names=class_labels)
# print(report)

# def plot_confusion_matrix(cm, classes,
#                         normalize=False,
#                         title='Confusion matrix',
#                         cmap=plt.cm.Blues):
   
#     #This function prints and plots the confusion matrix.
#     #Normalization can be applied by setting `normalize=True`
    
#     plt.imshow(cm, interpolation='nearest', cmap=cmap)
#     plt.title(title)
#     plt.colorbar()
#     tick_marks = np.arange(len(classes))
#     plt.xticks(tick_marks, classes, rotation=45)
#     plt.yticks(tick_marks, classes)

#     if normalize:
#         cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
#         print("Normalized confusion matrix")
#     else:
#         print('Confusion matrix, without normalization')

#     print(cm)

#     thresh = cm.max() / 2.
#     for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
#         plt.text(j, i, cm[i, j],
#             horizontalalignment="center",
#             color="white" if cm[i, j] > thresh else "black")

#     plt.tight_layout()
#     plt.ylabel('True label')
#     plt.xlabel('Predicted label')

# #Compute Confusion Matrix
# cnf_matrix = confusion_matrix(true_classes, predicted_classes)
# cm_plot_labels = ['bacterial_blight', 'cercospora_leaf_blight','downey_mildew', 'frogeye', 'non_disease', 'potassium_deficiency', 'soybean_rust', 'target_spot']
# np.set_printoptions(precision=2)
# #Plot non-normalized cm
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=cm_plot_labels, title='Confusion Matrix, Without Normalization')
# plt.savefig("without_normalized.png")

# #Plot normalized cm
# plt.figure()
# plot_confusion_matrix(cnf_matrix, classes=cm_plot_labels, normalize=True, title='Normalized Confusion Matrix')
# plt.savefig("normalized.png")


#Evaluate the performance of the model using test_generator
test_metrics = model.evaluate(test_generator, verbose=2, return_dict=True)
print('\nTest Accuracy:', test_metrics)

val_metrics = model.evaluate(valid_generator, verbose=2, return_dict=True)
print('\nValidation Accuracy:', val_metrics)

84/84 [==============================] - 46s 476ms/step
[[15  0 15]
 [ 7  0 14]
 [22  0 11]]
84/84 - 45s - loss: 1.1042 - accuracy: 0.3333 - 45s/epoch - 540ms/step

Test Accuracy: {'loss': 1.1042088270187378, 'accuracy': 0.3333333432674408}
5/5 - 50s - loss: 1.0674 - accuracy: 0.4247 - 50s/epoch - 10s/step

Validation Accuracy: {'loss': 1.0674052238464355, 'accuracy': 0.42465752363204956}


In [ ]:
import numpy as np
import os
import random
import tensorflow as tf
import ssl
from tensorflow import keras
from tensorflow.keras import layers, regularizers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model, Input
from tensorflow.compat.v1.keras.backend import set_session
from tensorflow.keras.backend import manual_variable_initialization
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
import itertools
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
from tensorflow import profiler
from tensorflow import keras
from keras_flops import get_flops
tf.enable_eager_execution()

model = keras.models.load_model('model_file_path')

# # Calculate FLOPs, may need to change batch size
flops = get_flops(model, batch_size=32)
print(f"FLOPS: {flops / 10 ** 9:.03} G")


import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.python.tools import freeze_graph
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2
from PIL import Image
import time
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.models import Model

model = tf.keras.models.load_model('model_file_path')

t00=time.time()
graph = tf.Graph()
graph_def = tf.compat.v1.GraphDef()

with graph.as_default():
    tf.import_graph_def(graph_def)


t01=time.time()
print("Iniialization time",t01-t00)

folder_path = os.path.abspath('images_file_path')
images = []
for img in os.listdir(folder_path):
    cwd = os.getcwd()
    files = os.listdir(cwd)
    print("Files in %r: %s" % (cwd, files))
    img = image.load_img(img, color_mode='rgb', target_size=(224,224,3))
    img = tf.keras.preprocessing.image.array_to_img(img)
    img = np.expand_dims(img, axis=0)
    img = img / 255
    # img = img.reshape((-1,) + img.shape)
    images.append(img)
images = np.vstack(images)
# classes = classifier.predict(images, batch_size=32)

print(img.dtype)
print(img.shape)


t0=time.time()

#Batch size has the model look at 32 images, so to get the average time for 1 image, divide the inference time by 32.
y_pred = model.predict(images, batch_size=32)
print(y_pred)
classname = y_pred[0]
print("Class: ",classname)

t1 = time.time()
print("Inference time", t1-t0)


In [ ]:
# upload saved model to google drive
# import shutil
# shutil.copy('saved_model/dense_1118.h5',"/content/drive/MyDrive")

In [ ]:
# %mkdir saved_model
# download saved model to 
# %cp /content/drive/MyDrive/dense_1118.hdf5 /content/saved_model

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import random
import tensorflow as tf
import itertools
import sklearn.metrics as metrics
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.densenet import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.densenet import DenseNet201
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger
from sklearn.metrics import classification_report, confusion_matrix, plot_confusion_matrix

#Setting seeds so that randomization is kept consistent.
np.random.seed(70) 
random.seed(70) 
tf.random.set_seed(70)

log_csv = CSVLogger('my_logs_robocrop.csv', separator=',', append = False)
callbacks_list = [log_csv]

img_height, img_width = (224,224)
batch_size = 32

train_data_dir = r"full_images_2021/train_full"
valid_data_dir = r"full_images_2021/test_full"
test_data_dir = r"full_images_2021/validation_full"

# A linha "preprocessing_function=preprocess_input" significa que uma
# função de pré-processamento será aplicada aos dados de entrada antes de 
#serem alimentados no modelo. O argumento "preprocess_input" é uma função
# pré-definida da biblioteca Keras (ou do seu provedor, como o TensorFlow) que 
#aplica algumas operações de pré-processamento comuns em imagens, como normalização
# de pixels, para melhorar o desempenho do modelo de aprendizado de máquina.




train_datagen = ImageDataGenerator(
    # preprocessing_function=preprocess_input,
    width_shift_range= 0.2, 
    height_shift_range=0.2,
    fill_mode="nearest",
    brightness_range=[0.9,1.1], 
    rotation_range =30, 
    vertical_flip = True,
    horizontal_flip = True,
    validation_split = 0.05,
    rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_height, img_width),
    batch_size = batch_size,
    class_mode = 'categorical'
    ) 

valid_generator = train_datagen.flow_from_directory(
    valid_data_dir, 
    target_size = (img_height, img_width),
    batch_size=batch_size,
    class_mode= 'categorical',
    ) 

test_generator = train_datagen.flow_from_directory(
    test_data_dir, 
    target_size = (img_height, img_width),
    batch_size=1,
    class_mode= 'categorical'
)
    # shuffle=False) 



Found 230 images belonging to 3 classes.
Found 73 images belonging to 3 classes.
Found 84 images belonging to 3 classes.


In [ ]:
model_path = '/content/drive/MyDrive/saved_model/best_val_acc.h5'
model = keras.models.load_model(model_path)

test_steps = len(test_generator)
Y_pred = model.predict(test_generator, steps= test_steps)
true_classes = test_generator.classes
predicted_classes = np.argmax(Y_pred, axis=1)

cm = confusion_matrix(true_classes, predicted_classes)
print("Confusion Matrix:\n", cm)

test_loss, test_accuracy = model.evaluate(test_generator, verbose=2)
print("\nTest Accuracy: {:.4f}".format(test_accuracy))

val_loss, val_accuracy = model.evaluate(valid_generator, verbose=2)
print("\nValidation Accuracy: {:.4f}".format(val_accuracy))


84/84 [==============================] - 49s 528ms/step
Confusion Matrix:
 [[ 9 10 11]
 [ 5  7  9]
 [16  3 14]]
84/84 - 43s - loss: 0.1878 - accuracy: 0.9286 - 43s/epoch - 516ms/step

Test Accuracy: 0.9286
3/3 - 50s - loss: 0.1604 - accuracy: 0.9589 - 50s/epoch - 17s/step

Validation Accuracy: 0.9589
